In [1]:
%load_ext rpy2.ipython

In [2]:
from IPython.display import IFrame

# Molecular epidemiology of viruses

## Introduction

- Viruses, especially RNA viruses, are highly genetically variable
- This variation allows us to build high-resolution phylogenetic trees
- Combined with dates of sampling, we can put a timescale on these trees
- Integrating the time-stamped trees with mathematical models we can infer:
    - Population size changes over time
    - Migration patterns between subpopulations

## Steps

- **Day 1. Phylogenetics**
    - Obtaining sequence data
    - Multiple sequence alignment
    - Trimming and editing, if needed
    - Screening for recombination
    - Choosing an evolutionary model
    - Reconstructing a maximum likelihood tree
- **Day 2. Phylodynamics**
    - Obtaining a time-tree
    - Inference of effective population size
    - Phylogeography

## Software used

- [gbmunge](https://github.com/sdwfrost/gbmunge)
- [MAFFT](https://mafft.cbrc.jp/alignment/software/)
- [Seaview](http://doua.prabi.fr/software/seaview) or [AliView](http://ormbunkar.se/aliview/)
- [IQ-TREE](http://www.iqtree.org/)
- [FigTree](http://tree.bio.ed.ac.uk/software/figtree/)
- [TempEst](http://tree.bio.ed.ac.uk/software/tempest/)
- [R](https://www.r-project.org/) and [RStudio](https://www.rstudio.com/)
- R packages:
    [treedater](https://github.com/emvolz/treedater)
    [skyspline](https://github.com/emvolz-phylodynamics/skyspline/)
    [phyland](https://github.com/emvolz-phylodynamics/phyland)
- [BEAST2](http://www.beast2.org/)

## RStudio

- Interface to the R programming language
- We will use R to
    - Manipulate data
    - Fit coalescent models
    - Plotting
- Also has:
    - A terminal for running command-line programs
    - A text editor for viewing text files

## Example datasets

- Demonstration dataset: Ebola virus sequences sampled from humans in West Africa (>1000)
    - See [Dudas et al. Nature (2017)](http://dx.doi.org/10.1038/nature22040)
- Tutorial dataset: MERS coronavirus sequences from camels and humans in Saudi Arabia (>200)
    - See [Dudas et al. biorXiv (2017)](http://dx.doi.org/10.1101/173211)

## Questions

- **When did the epidemic start?**
    - Generate a time-stamped phylogeny
    - We can calculate the time to the most recent common ancestor
- **Is the epidemic expanding, stable, or declining?**
    - We can fit coalescent models to the data
- **Where did the epidemic start?**
    - Use *structured* coalescent models
- These are phylodynamic questions, but first we need to obtain and process sequence data

# Obtaining data

## Data formatting

- Sequence data
- Metadata
    - We require *collection date*
    - We may also be interested in:
        - Country of sampling
        - Host
- Often missing metadata
    - Collection date may be missing
    - Date may be approximate, e.g.
        - 2013
        - 2013-01

## Sequence data formatting

- Many programs are fussy about data formats
- The following FASTA format usually works
    - Accession/sequence identifier
    - Collection date


```
>JX869059_2012-06-13
gatttaagtgaatagcttggctatctcacttcccctcgttctcttgcagaactttgattttaac...
```

## Metadata formatting

- Metadata in a tab-separated file

```
               name         host         country          date
KF186564_2013-05-01	Homo sapiens	Saudi Arabia	2013-05-01
KF186565_2013-04-22	Homo sapiens	Saudi Arabia	2013-04-22
```

## Obtain sequences

- There are several databases where sequences can be downloaded
    - GenBank
    - Virus Variation Resource
    - GISAID (influenza)
    - Los Alamos Databases (HIV, HCV, haemorhagic fever databases)

## Virus variation resource

- [https://www.ncbi.nlm.nih.gov/genome/viruses/variation/](https://www.ncbi.nlm.nih.gov/genome/viruses/variation/)
- Annotated databases of:
    - Influenza
    - Rotavirus
    - Dengue
    - West Nile virus
    - Ebola virus
    - Zika virus
    - MERS Coronavirus
- Offers a convenient search and download interface

In [2]:
IFrame('https://www.ncbi.nlm.nih.gov/genomes/VirusVariation/Database/nph-select.cgi?&taxid=1335626', 800, 350)

## Searching for Ebola sequences

Link [here](https://www.ncbi.nlm.nih.gov/genomes/VirusVariation/Database/nph-select.cgi?cmd=show_builder&country=1&download-select=fN&fday=1&fmonth=12&fyear=2013&genregion=any&go=database&host=Human&isolation=any&query_1_count=1902&query_1_count_genome_sets=0&query_1_country=1&query_1_fday=1&query_1_fmonth=12&query_1_fyear=2013&query_1_genregion=any&query_1_host=Human&query_1_isolation=any&query_1_line=on&query_1_line_num=1&query_1_query_key=1&query_1_searchin=sequence&query_1_sequence=N&query_1_species=Z&query_1_srcfilter_env_sample=include&query_1_srcfilter_labs=include&query_1_srcfilter_vac_strain=include&query_1_taxid=186536&query_1_tday=1&query_1_tmonth=7&query_1_tyear=2016&searchin=sequence&sequence=N&showfilters=true&species=Z&srcfilter_env_sample=include&srcfilter_labs=include&srcfilter_vac_strain=include&taxid=186536&tday=1&tmonth=7&tyear=2016)

## GenBank

- [https://www.ncbi.nlm.nih.gov/genbank/](https://www.ncbi.nlm.nih.gov/genbank/)
- The NIH genetic sequence database, an annotated collection of all publicly available DNA sequences
- Part of the International Nucleotide Sequence Database Collaboration
    - GenBank
    - DNA DataBank of Japan (DDBJ)
    - European Nucleotide Archive (ENA)

## GenBank format

```
LOCUS       JX869059               30119 bp    RNA     linear   VRL 04-DEC-2012
DEFINITION  Human betacoronavirus 2c EMC/2012, complete genome.
ACCESSION   JX869059
VERSION     JX869059.2
KEYWORDS    .
SOURCE      Human betacoronavirus 2c EMC/2012
  ORGANISM  Human betacoronavirus 2c EMC/2012
            Viruses; ssRNA viruses; ssRNA positive-strand viruses, no DNA
            stage; Nidovirales; Coronaviridae; Coronavirinae; Betacoronavirus.
...
```

## FASTA format

```
>JX869059.2 Human betacoronavirus 2c EMC/2012, complete genome
GATTTAAGTGAATAGCTTGGCTATCTCACTTCCCCTCGTTCTCTTGCAGAACTTTGATTTTAACGAACTT
AAATAAAAGCCCTGTTGTTTAGCGTATCGTTGCACTTGTCTGGTGGGATTGTGGCATTAATTTGCCTGCT
CATCTAGGCAGTGGACATATGCTCAACACTGGGTATAATTCTAATTGAATACTATTTTTCAGTTAGAGCG
TCGTGTCTCTTGTACGTCTCGGTCACAATACACGGTTTCGTCCGGTGCGTGGCAATTCGGGGCACATCAT
GTCTTTCGTGGCTGGTGTGACCGCGCAAGGTGCGCGCGGTACGTATCGAGCAGCGCTCAACTCTGAAAAA
CATCAAGACCATGTGTCTCTAACTGTGCCACTCTGTGGTTCAGGAAACCTGGTTGAAAAACTTTCACCAT
GGTTCATGGATGGCGAAAATGCCTATGAAGTGGTGAAGGCCATGTTACTTAAAAAGGAGCCACTTCTCTA
```

## Converting formats

- If the virus you are working on is only available in GenBank, then you will have to convert the sequence data into the correct format
- I have written a small C program that takes a GenBank formatted sequence file and generates:
    - A FASTA formatted file with names "accession_date"
    - A tab-delimited file with host, country, and date of sampling

## Searching GenBank

- It is a good idea to start with looking for the taxonomy ID of the pathogen
    - [https://www.ncbi.nlm.nih.gov/taxonomy](https://www.ncbi.nlm.nih.gov/taxonomy)
- Search for the pathogen of interest
    - e.g. Zaire ebolavirus
- Click on the taxon name
- Click on the link to the nucleotide database
- Select, using filters on the right

## Near full-length genomes of Ebola virus

```
txid186538[Organism:exp] AND
(ddbj_embl_genbank[filter] AND
("18000"[SLEN] : "19000"[SLEN]) AND
("2013/12/01"[PDAT] : "2017/09/30"[PDAT]))
```

## Exercise: downloading from Virus Variation

1. Download sequences and metadata of MERS Coronavirus from the Virus Variation Resource with the following filters:
    - Nucleotide sequences
    - Host: camels or humans
    - Country: Saudi Arabia
    - Link [here](https://www.ncbi.nlm.nih.gov/genomes/VirusVariation/Database/nph-select.cgi?cmd=show_builder&country=Saudi%20Arabia&download-select=fN&genregion=any&go=database&host=Camel&host=Human&isolation=any&query_1_count=471&query_1_count_genome_sets=0&query_1_country=Saudi%20Arabia&query_1_genregion=any&query_1_host=Camel&query_1_host=Human&query_1_isolation=any&query_1_line=on&query_1_line_num=1&query_1_query_key=1&query_1_searchin=sequence&query_1_sequence=N&query_1_srcfilter_env_sample=include&query_1_srcfilter_labs=include&query_1_srcfilter_vac_strain=include&query_1_taxid=1335626&searchin=sequence&sequence=N&showfilters=true&srcfilter_env_sample=include&srcfilter_labs=include&srcfilter_vac_strain=include&taxid=1335626)
2. Select sequences greater than 29,000 base pairs long
3. Format the sequence names to be "accession_date"
4. Download nucleotide sequences in FASTA format, calling it `mers.fas`. Open up the file in a text editor.
5. Download the result set in tab-delimited format, calling it `mers.txt`. Open up in e.g. Excel or OpenOffice to see the results

## Renaming and filtering the sequences

- The sequence names will not be accepted by many software programs, as they contain certain characters
    - We need to rename sequences such as `KT805988_2015/05/10` to `KT805988_2015/05/10`
- We are also going to filter our sequences to near-full length genomes, by choosing sequences only sequences 29,000 base pairs long or greater
- I have given you an R script, `trimmers.R` to do this for you

```r
library(ape)

ifn <- "data/mers.fas"
ofn <- "data/mers.fas.long"
thresh <- 29000

s <- read.dna(ifn,format="fasta",as.matrix=FALSE)
nm <- names(s)
names(s) <- gsub("/","-",nm,fixed=TRUE)
l <- unlist(lapply(s,length))
s <- s[l>=thresh]

write.dna(s,ofn,format="fasta",nbcol=-1,colsep="")
```

## Aside: downloading from GenBank

1. Download MERS-CoV sequences from GenBank between 29,000 and 31,000 base pairs long and convert into FASTA sequences and metadata
    - You should end up with a search term as follows:

```
txid1335626[Organism:exp] AND (ddbj_embl_genbank[filter] AND ("29000"[SLEN] : "31000"[SLEN]))
```

2. Convert to FASTA and tabular format using `gbmunge`

```sh
gbmunge -i sequence.gb -f sequence.fas -o sequence.txt -t
```

## Summary

- You should now have
    - Sequences in FASTA format, with names in the correct format
    - Associated metadata

# Multiple sequence alignment

## Align sequences

- Sequences need to be *aligned* in order to perform downstream analyses
- Multiple sequence alignment is an NP-hard problem, so heuristics are involved
    - Different alignment programs give different results
- In our work, we have found the program MAFFT to give good accuracy with high speed

## MAFFT usage

<small>

```sh
High speed:
  % mafft.bat in > out
  % mafft.bat --retree 1 in > out (fast)

High accuracy (for <~200 sequences x <~2,000 aa/nt):
  % mafft.bat --maxiterate 1000 --localpair  in > out (% linsi in > out is also ok)
  % mafft.bat --maxiterate 1000 --genafpair  in > out (% einsi in > out)
  % mafft.bat --maxiterate 1000 --globalpair in > out (% ginsi in > out)

If unsure which option to use:
  % mafft.bat --auto in > out

--op # :         Gap opening penalty, default: 1.53
--ep # :         Offset (works like gap extension penalty), default: 0.0
--maxiterate # : Maximum number of iterative refinement, default: 0
--clustalout :   Output: clustal format, default: fasta
--reorder :      Outorder: aligned, default: input order
--quiet :        Do not report progress
--thread # :     Number of threads (if unsure, --thread -1)
```

</small>

## Exercise: align sequences using MAFFT

- Align the FASTA formatted MERS-CoV sequences using MAFFT

```sh
mafft.bat --auto --thread -1 mers.fas.long > mers.fas.long.aligned
```

# Quality control

## Quality control

- Once you have aligned sequences, it is important to check the alignment visually using an alignment editor
    - Especially as downstream analyses are often computationally intensive
- Common manipulations:
    - Trimming
    - Exclusion of low quality/divergent sequences
    - Putting sequences into reading frame

## Exercise: visualise alignment

1. Use Seaview to visualise the alignment
    - In this case, no changes should be necessary

## Screening for recombination

- Recombination leads to different parts of the alignment having different evolutionary histories
- Building a single phylogeny from the entire alignment can be misleading
- It is advisable to screen for recombination using tools such as [RDP4](http://web.cbio.uct.ac.za/~darren/rdp.html)
- Strategies:
    - Removal of recombinant sequences
    - Only use a non-recombinant part of the alignment
    - Split recombinant sequences into non-recombinant segments
- Due to time constraints, we will assume that our sequences are free of recombination

# Phylogenetic reconstruction

## Reconstructing a phylogeny

- A phylogeny is a tree that represents the evolutionary relationships between a set of sequences
- There are multiple methods to reconstruct a phylogeny
    - Parsimony
    - Distance based methods
    - Likelihood based methods:
        - **Maximum likelihood**
        - Bayesian

## Maximum likelihood inference of phylogeny

- For ML inference we need to assume a model for how the sequence evolves
- For nucleotide sequences, this is the rate at which one base is substituted for another

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/2842b61fd25a43260e6e01a016ecdfb302312651)

- See [https://en.wikipedia.org/wiki/Models_of_DNA_evolution](https://en.wikipedia.org/wiki/Models_of_DNA_evolution) for more information on models

## General time reversible (GTR) model

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/a3f55f3a07a6821f3082c64027ff66fb86bf9460)

## Modeling rate variation

- In addition to varying rates of substitution between different bases, there can also be variation across sites in the alignment
    - Conserved positions, e.g. due to functional constraints
    - Highly variable positions, e.g. due to immune selection
- We can allow the overall substitution rate to vary across positions according to some probability distribution
- This distribution is often discretised into categories for computational reasons
- Common choices
    - Gamma distribution
    - Free rates distribution

## Choosing substitution models

- Choosing a model means striking a balance:
    - More parameters = better fit to the data
    - Less parameters = simpler model
- We can compare different models by computing the fit (likelihood) and the complexity (the number of parameters) and choosing the 'best' one
- Two common criteria (smaller is better):
    - Akaike's Information Criterion (AIC)
    - Bayesian Information Criterion (BIC)
        - Tends to favour simpler models than AIC

## IQ-TREE

- [IQ-TREE](http://www.iqtree.org/) is a program that can perform:
    - Tests of substitution models
    - Phylogenetic reconstruction
- Fast, flexible, and is actively developed
- We will use the development (1.6) version

## Exercise: getting a rough tree

- Run IQ-TREE and take a look at the output files

Usage:

```sh
iqtree -s mers.fas.long.aligned -m TN93 -pre mers_prelim -nt 2 -fast
```

- `-s`: sequence file in FASTA
- `-m`: model of substitution used (here, TN93)
- `-pre`: prefix used for output files
- `-nt`: number of threads used
- `-fast`: uses fast option (more later)



## Exercise: visualising the tree in Seaview

- Open up the resulting tree (should be called `mers_prelim.treefile`) in Seaview
- Do you see any outlying sequences?
- If you also open up the sequence file, `mers.fas.long.aligned`, you can select sequences by selecting them on the tree
    - Useful to exclude outliers

## Exercise: choosing a model

1. Once you have a 'final' alignment, use IQTREE to choose a substitution model

Usage:

```sh
iqtree -s <FASTA file> -m MF -pre <prefix> -nt <number of threads>
```

Options to restricted set of models, e.g.

```sh
-mset HKY,TN,GTR -mrate E,G,R
```

## Output of model selection

- For example, for Ebola sequences

```sh
 No. Model         -LnL         df  AIC          AICc         BIC
  1  GTR           68171.876    3765 143873.753   145672.660   173540.850
  2  GTR+R2        67095.370    3767 141724.740   143525.788   171407.597
  3  GTR+R3        66879.788    3769 141297.577   143100.766   170996.192
  4  GTR+R4        66800.124    3771 141142.248   142947.580   170856.623
  5  GTR+R5        66765.734    3773 141077.467   142884.944   170807.602
  6  GTR+R6        66753.528    3775 141057.056   142866.679   170802.950
  7  GTR+R7        66750.018    3777 141054.035   142865.806   170815.689
```

- Take a look at your output from model selection; which model is 'best'?

## Reconstruct phylogeny

- Once we have a model (which uses a fast, approximate phylogeny), we can reconstruct a phylogeny
- Tree search has to be done heuristically, as there are too many trees to test all of them
- In maximum likelihood
    - Start with a rough tree
    - Change the tree and compare the likelihood of the new tree with the old
    - Keep changing the tree until the likelihood does not improve

## Tree rearrangements

- Three basic rearrangements in wide use:
    - Nearest neighbour interchange
    - Subtree pruning and regrafting
    - Tree bisection and reconnection
- More information at [https://en.wikipedia.org/wiki/Tree_rearrangement](https://en.wikipedia.org/wiki/Tree_rearrangement)

## Nearest neighbour interchange (NNI)

![](https://upload.wikimedia.org/wikipedia/commons/5/5f/NNI.svg)

## Subtree pruning and regrafting (SPR)

![](https://upload.wikimedia.org/wikipedia/commons/3/3b/SPR.svg)

## Tree bisection and reconnection (TBR)

![](https://upload.wikimedia.org/wikipedia/commons/0/0d/TBR.svg)

## Tree reconstruction with IQ-TREE

- IQ-TREE uses a stochastic search approach to find the 'best' tree
- Additional options:
    - `-fast`: performs fast but approximate search
    - `-allnni`: performs more thorough NNI search

## Exercise: reconstruct phylogeny

1. Use IQ-TREE to generate a rough phylogeny using the `-fast` option

Usage:

```sh
iqtree -s <FASTA file> -m <model> -pre <prefix> -nt <threads> -fast
```

## Visualise tree

- Once we have a tree, we need to visualise it
- Things to look for:
    - Temporal signal: sequences sampled later are more divergent
    - Outlier sequences
        - Recombinants
        - Alignment errors
        - Hypermutation
        - Arrested evolution

## TempEst

- [TempEst](http://tree.bio.ed.ac.uk/software/tempest/) is a program that roots a phylogeny in order to maximise the association of sampling times with distance to the root of the tree
- This makes it easier to spot divergent sequences

## Exercise: visualising the tree

1. Use TempEst to visualise the maximum likelihood tree from IQ-TREE
    - Read in tree
    - Guess dates of sampling from sequence headers
    - Infer root
2. Questions:
    - When is the most recent common ancestor (TMRCA)?
    - Why might this method not be the most robust way of finding TMRCA?
    - How strong is the temporal signal?

## Addendum: significant clusters

- Maximum likelihood gives you a single tree
- Like all inferences, there is error around this tree
- How do we capture this error?

## Nonparametric bootstrapping

- Idea: produce replicate datasets by resampling the original data
    - Generate new alignments from sampling columns from the old alignment
- Run the analysis on these replicate datasets
- Often, a *consensus tree* is generated from the independent runs
    - Contains how often a particular group of sequences in the original tree clusters together in the bootstrap replicates

## Other approximate bootstrap approaches

- Approximate likelihood ratio test (aLRT)
    - Tests whether a branch can be collapsed to zero or not
- UFboot/UFboot2 (ultrafast bootstrap approximation)
    - Uses trees found in maximum likelihood search and resampling of partial likelihoods
- Simple bootstrap sampling, aLRT, and UFboot/UFboot2 all available in IQ-TREE

## Exercise: nonparametric bootstrapping

- To add simple bootstrap sampling, add `-b` followed by the number of bootstrap replicates to the command line.

e.g.

```sh
iqtree -s mers.fas.long.aligned -m GTR+R2 -pre mers_boot -nt 2 -fast -b 100
```

- This will generate two additional files:
    - `*.boottrees`: bootstrap trees
    - `*.contree`: consensus tree

## Exercise: visualising branch supports

- Open up the consensus tree in FigTree.

## Summary so far

- Obtained sequence data and metadata in the appropriate formats
- Aligned sequences and checked
- Chosen an evolutionary model
- Reconstructed a tree
- Rooted the tree based on sampling times
- Next steps: phylodynamics